## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-31-14-53-32 +0000,nypost,Newly-released video shows horrific moment Arm...,https://nypost.com/2025/07/31/us-news/video-sh...
1,2025-07-31-14-52-54 +0000,nypost,Nancy Pelosi blows up at Jake Tapper over insi...,https://nypost.com/2025/07/31/us-news/nancy-pe...
2,2025-07-31-14-49-11 +0000,nyt,Witkoff Plans Visit to Gaza to See Hunger Crisis,https://www.nytimes.com/2025/07/31/world/middl...
3,2025-07-31-14-48-47 +0000,startribune,Stearns Bank says Duluth Central High School s...,https://www.startribune.com/stearns-bank-incli...
4,2025-07-31-14-45-09 +0000,bbc,Hotel tycoon reveals Heathrow expansion proposal,https://www.bbc.com/news/articles/cd6nvzjvwgqo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2364/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
85,trump,71
145,tariffs,20
87,trade,19
158,new,17
28,gaza,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
214,2025-07-30-22-42-00 +0000,nypost,Trump announces South Korea trade deal with $3...,https://nypost.com/2025/07/30/us-news/trump-an...,153
81,2025-07-31-10-33-13 +0000,cbc,U.S. to set 15% tariff on South Korean imports...,https://www.cbc.ca/news/world/trump-tariffs-so...,140
337,2025-07-30-16-33-22 +0000,bbc,Trump to hit India with 25% tariffs - plus 'pe...,https://www.bbc.com/news/articles/cz71w1dgl84o...,138
204,2025-07-30-23-08-32 +0000,cbc,Trump slams Canada's plan to recognize Palesti...,https://www.cbc.ca/news/politics/us-canada-tra...,137
47,2025-07-31-12-28-38 +0000,wapo,Trump puts trade deal in doubt after Canada ba...,https://www.washingtonpost.com/politics/2025/0...,136


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
214,153,2025-07-30-22-42-00 +0000,nypost,Trump announces South Korea trade deal with $3...,https://nypost.com/2025/07/30/us-news/trump-an...
282,93,2025-07-30-20-19-00 +0000,wsj,Former Vice President Kamala Harris said Wedne...,https://www.wsj.com/politics/elections/kamala-...
17,72,2025-07-31-14-01-19 +0000,nypost,Fed’s preferred inflation gauge ticks up as ta...,https://nypost.com/2025/07/31/business/feds-pr...
218,54,2025-07-30-22-31-15 +0000,nypost,Witkoff could visit Gaza aid sites during Isra...,https://nypost.com/2025/07/30/world-news/witko...
0,49,2025-07-31-14-53-32 +0000,nypost,Newly-released video shows horrific moment Arm...,https://nypost.com/2025/07/31/us-news/video-sh...
329,47,2025-07-30-17-01-29 +0000,wapo,NYC mass killing raises new security concerns ...,https://www.washingtonpost.com/business/2025/0...
79,47,2025-07-31-10-37-06 +0000,bbc,Canada follows France and UK with plan to reco...,https://www.bbc.com/news/articles/ceqyx35d9x2o...
323,41,2025-07-30-17-43-00 +0000,wsj,Tsunami Waves Reach U.S. After Powerful Russia...,https://www.wsj.com/world/tsunami-watch-warnin...
341,36,2025-07-30-16-23-00 +0000,wsj,Two top Senate Republicans laid out a plan Wed...,https://www.wsj.com/politics/policy/us-weapons...
136,36,2025-07-31-05-55-25 +0000,nypost,Delta flight diverted after being hit by turbu...,https://nypost.com/2025/07/31/us-news/delta-fl...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
